In [1]:
!pip install cpi

  Using cached cpi-1.0.0-py2.py3-none-any.whl (29.0 MB)


In [24]:
import numpy as np
import pandas as pd
from siuba import *

from calitp import to_snakecase

import _dla_utils

In [2]:
import cpi

In [3]:
df= pd.read_parquet("dla_df.parquet")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20117 entries, 0 to 20446
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   location                20117 non-null  object        
 1   prefix                  20117 non-null  object        
 2   project_no              20117 non-null  object        
 3   agency                  20117 non-null  object        
 4   prepared_date           20102 non-null  datetime64[ns]
 5   submit__to_hq_date      19945 non-null  datetime64[ns]
 6   hq_review_date          19914 non-null  datetime64[ns]
 7   submit_to_fhwa_date     19896 non-null  datetime64[ns]
 8   to_fmis_date            19896 non-null  datetime64[ns]
 9   fed_requested           20116 non-null  float64       
 10  ac_requested            20117 non-null  float64       
 11  total_requested         20116 non-null  float64       
 12  status_comment          20117 non-null  object

In [6]:
df= df>>select(_.primary_agency_name, _.dist, _.prepared_date, _.prefix, _.prepared_y, _.fed_requested, _.ac_requested, _.total_requested)
df

,primary_agency_name,dist,prepared_date,prefix,prepared_y,fed_requested,ac_requested,total_requested
0,Humboldt County,1,2018-12-18,BPMP,"2,018.00",0.00,0.00,0.00
1,Mendocino County,1,2018-12-17,ER,"2,018.00","11,508.00",0.00,"13,000.00"
2,Humboldt County,1,2018-12-07,ER,"2,018.00","45,499.64",0.00,"51,394.58"
3,Sacramento County,3,2018-12-11,CML,"2,018.00","207,002.00",0.00,"247,002.00"
4,Sacramento County,3,2018-12-05,CML,"2,018.00",0.00,"5,680,921.00","5,702,041.00"
...,...,...,...,...,...,...,...,...
20441,San Francisco Bay Area Rapid Transit District,4,NaT,FTACRRS,NaN,0.00,0.00,0.00
20442,Marin County Transit District,4,NaT,FTASTPL,NaN,0.00,0.00,0.00
20443,Santa Clara Valley Transportation Authority,4,NaT,FTASTPL,NaN,0.00,0.00,0.00
20444,Alameda - Contra Costa Transit District,4,NaT,FTASTPL,NaN,0.00,0.00,0.00


In [7]:
sorted(df.prepared_y.unique())

[2010.0,
 2011.0,
 2012.0,
 2013.0,
 2014.0,
 2015.0,
 2016.0,
 2017.0,
 2018.0,
 nan,
 2019.0,
 2020.0,
 2021.0]

In [8]:
df.tail(1)

,primary_agency_name,dist,prepared_date,prefix,prepared_y,fed_requested,ac_requested,total_requested
20446,Caltrans,11,NaT,HSIPL,NaN,"351,200.00",0.00,"351,200.00"


In [22]:
cpi.inflate(351200, 2018)

378980.5302122203

In [20]:
# Inflation table
def inflation_table(base_year):
    cpi.update()
    series_df = cpi.series.get(area="U.S. city average").to_dataframe()
    inflation_df = (series_df[series_df.year >= 2008]
           .pivot_table(index='year', values='value', aggfunc='mean')
           .reset_index()
          )
    denominator = inflation_df.value.loc[inflation_df.year==base_year].iloc[0]

    inflation_df = inflation_df.assign(
        inflation = inflation_df.value.divide(denominator)
    )
    
    return inflation_df

In [19]:
def adjust_prices(df):
    
    cols =  ["total_requested",
           "fed_requested",
           "ac_requested"]
    
    ##get cpi table 
    cpi = inflation_table(2021)
    cpi.update
    cpi = (cpi>>select(_.year, _.value))
    cpi_dict = dict(zip(cpi['year'], cpi['value']))
    
    
    for col in cols:
        multiplier = df["prepared_y"].map(cpi_dict)  
    
        ##using 270.97 for 2021 dollars
        df[f"adjusted_{col}"] = ((df[col] * 270.97) / multiplier)
    return df


In [16]:
adjust_prices(df)

,primary_agency_name,dist,prepared_date,prefix,prepared_y,fed_requested,ac_requested,total_requested,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested
0,Humboldt County,1,2018-12-18,BPMP,"2,018.00",0.00,0.00,0.00,0.00,0.00,0.00
1,Mendocino County,1,2018-12-17,ER,"2,018.00","11,508.00",0.00,"13,000.00","14,028.33","12,418.31",0.00
2,Humboldt County,1,2018-12-07,ER,"2,018.00","45,499.64",0.00,"51,394.58","55,460.01","49,098.77",0.00
3,Sacramento County,3,2018-12-11,CML,"2,018.00","207,002.00",0.00,"247,002.00","266,540.45","223,376.35",0.00
4,Sacramento County,3,2018-12-05,CML,"2,018.00",0.00,"5,680,921.00","5,702,041.00","6,153,086.12",0.00,"6,130,295.48"
...,...,...,...,...,...,...,...,...,...,...,...
20441,San Francisco Bay Area Rapid Transit District,4,NaT,FTACRRS,NaN,0.00,0.00,0.00,NaN,NaN,NaN
20442,Marin County Transit District,4,NaT,FTASTPL,NaN,0.00,0.00,0.00,NaN,NaN,NaN
20443,Santa Clara Valley Transportation Authority,4,NaT,FTASTPL,NaN,0.00,0.00,0.00,NaN,NaN,NaN
20444,Alameda - Contra Costa Transit District,4,NaT,FTASTPL,NaN,0.00,0.00,0.00,NaN,NaN,NaN


In [17]:
df>>filter(_.prepared_y==2015)

,primary_agency_name,dist,prepared_date,prefix,prepared_y,fed_requested,ac_requested,total_requested,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested
1010,Lake Elsinore,8,2015-12-02,BRLS,"2,015.00","1,639,576.00",0.00,"1,852,000.00","2,117,301.46","1,874,447.44",0.00
2508,Humboldt County,1,2015-10-12,ER,"2,015.00","1,329,549.14",0.00,"1,501,805.00","1,716,940.56","1,520,008.82",0.00
3219,Manhattan Beach,7,2015-10-15,HPLUL,"2,015.00","481,250.00",0.00,"481,250.00","550,189.70","550,189.70",0.00
3658,Palm Springs,8,2015-12-07,BRLS,"2,015.00","330,217.00",0.00,"373,000.00","426,432.75","377,521.02",0.00
4858,Alameda County,4,2015-10-19,HPLUL,"2,015.00",-0.04,0.00,"22,718.91","25,973.42",-0.05,0.00
...,...,...,...,...,...,...,...,...,...,...,...
10256,Davis,3,2015-01-02,STPL,"2,015.00",0.00,0.00,"16,912.51","19,335.25",0.00,0.00
10257,San Carlos,4,2015-01-02,HSIP,"2,015.00",0.00,0.00,"12,075.00","13,804.76",0.00,0.00
10258,Campbell,4,2015-01-02,CML,"2,015.00",0.00,0.00,"182,395.73","208,524.16",0.00,0.00
10259,Pleasant Hill,4,2015-01-02,STPLX,"2,015.00","-11,726.26",0.00,"-13,245.89","-15,143.38","-13,406.06",0.00


In [18]:
#checking to see if it works. using first row
cpi.inflate(1852000.00, 2015)

2117301.4593889895

In [25]:
df>>filter(_.prepared_y== 2014)

,primary_agency_name,dist,prepared_date,prefix,prepared_y,fed_requested,ac_requested,total_requested,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested
8246,Coachella,8,2014-09-25,HPLULN,"2,014.00","719,920.75",0.00,"1,968,000.00","2,252,587.75","824,026.76",0.00
8257,Palm Springs,8,2014-07-03,HPLUL,"2,014.00","399,902.72",0.00,"1,130,683.00","1,294,188.35","457,731.69",0.00
8366,Richmond,4,2014-12-19,CML,"2,014.00",0.00,0.00,"1,130,041.00","1,293,453.51",0.00,0.00
8418,Monterey Park,7,2014-08-14,STPL,"2,014.00","1,036,867.00",0.00,"1,057,741.00","1,210,698.38","1,186,805.84",0.00
8488,Moreno Valley,8,2014-12-10,STPL,"2,014.00","1,084,000.00",0.00,"2,008,167.00","2,298,563.20","1,240,754.63",0.00
...,...,...,...,...,...,...,...,...,...,...,...
12988,Moreno Valley,8,2014-01-06,BRLS,"2,014.00","-131,768.16",0.00,"28,508.61","32,631.17","-150,822.84",0.00
12989,Santa Clarita,7,2014-01-06,BHLS,"2,014.00",-289.94,0.00,-327.50,-374.86,-331.87,0.00
12998,Ridgecrest,9,2014-01-03,STPL,"2,014.00",-198.00,0.00,"20,987.53","24,022.49",-226.63,0.00
13001,Solano County,4,2014-01-02,CML,"2,014.00","60,000.00",0.00,"67,774.00","77,574.64","68,676.46",0.00


In [26]:
#checking to see if it works. using first row
cpi.inflate(1968000, 2014)

2252589.2133008926